# 🔢 Project 1: The Vector Search Engine

**Objective:** Build a semantic search system from scratch to understand how AI Agents "retrieve" knowledge.

## 📖 What You'll Learn

- How text is converted to numerical vectors (embeddings)
- What cosine similarity measures and why it matters
- How to implement nearest neighbor search manually
- The foundation of RAG (Retrieval Augmented Generation) systems

## 🎯 Learning Goals

By the end of this notebook, you'll understand:
1. Vector representations of text
2. Mathematical similarity measures
3. How semantic search differs from keyword search
4. The fundamentals behind vector databases

## Setup and Imports

In [ ]:
# Install required packages (run once)
# !pip install numpy pandas scikit-learn sentence-transformers matplotlib

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

# For reproducibility
np.random.seed(42)

## Task 1: Load a Text Dataset

We'll create a simple knowledge base about AI and Machine Learning.

In [ ]:
# Sample documents - a mini knowledge base about AI/ML
documents = [
    "Machine learning is a subset of artificial intelligence that focuses on algorithms that improve through experience.",
    "Deep learning uses neural networks with multiple layers to learn hierarchical representations of data.",
    "Natural language processing enables computers to understand, interpret, and generate human language.",
    "Reinforcement learning involves agents learning to make decisions by interacting with an environment.",
    "Computer vision allows machines to interpret and understand visual information from the world.",
    "Supervised learning trains models on labeled data to make predictions on new, unseen data.",
    "Unsupervised learning finds patterns in data without explicit labels or targets.",
    "Transfer learning leverages pre-trained models to solve new but related tasks efficiently.",
    "Gradient descent is an optimization algorithm that minimizes loss functions by iteratively adjusting parameters.",
    "Transformers are neural network architectures that use self-attention mechanisms for sequence processing.",
]

print(f"📚 Loaded {len(documents)} documents into our knowledge base\n")
for i, doc in enumerate(documents, 1):
    print(f"{i}. {doc[:80]}...")

## Task 2: Convert Text to Vectors

We'll use a pre-trained sentence embedding model to convert text into dense vectors.

**Key Concept:** Each document becomes a point in high-dimensional space. Similar documents are close together.

In [ ]:
# Load a pre-trained embedding model
# We're using 'all-MiniLM-L6-v2' - a small, fast model that produces 384-dimensional vectors
model = SentenceTransformer('all-MiniLM-L6-v2')

print("🔄 Converting documents to vectors...\n")

# Encode all documents into vectors
document_embeddings = model.encode(documents)

print(f"✅ Created embeddings with shape: {document_embeddings.shape}")
print(f"   - Number of documents: {document_embeddings.shape[0]}")
print(f"   - Embedding dimensions: {document_embeddings.shape[1]}")
print(f"\n📊 First document vector (first 10 dimensions): {document_embeddings[0][:10]}")

## Task 3: Implement Cosine Similarity from Scratch

**Formula:** 
$$\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \times \|B\|}$$

Where:
- $A \cdot B$ is the dot product
- $\|A\|$ is the magnitude (L2 norm) of vector A

In [ ]:
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    Compute cosine similarity between two vectors.
    
    Args:
        vec1: First vector (1D numpy array)
        vec2: Second vector (1D numpy array)
    
    Returns:
        Similarity score between -1 and 1 (1 = identical, 0 = orthogonal, -1 = opposite)
    """
    # TODO: Implement this function using NumPy
    # Hint: Use np.dot() for dot product and np.linalg.norm() for magnitude
    
    # Step 1: Calculate dot product
    dot_product = np.dot(vec1, vec2)
    
    # Step 2: Calculate magnitudes
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    
    # Step 3: Calculate cosine similarity
    similarity = dot_product / (magnitude_vec1 * magnitude_vec2)
    
    return similarity

# Test the function
test_vec1 = np.array([1, 2, 3])
test_vec2 = np.array([2, 4, 6])  # Perfectly aligned (scaled version)
test_vec3 = np.array([1, 0, 0])  # Orthogonal to test_vec1

print("🧪 Testing cosine similarity function:")
print(f"   Similarity between [1,2,3] and [2,4,6]: {cosine_similarity(test_vec1, test_vec2):.4f} (expect ~1.0)")
print(f"   Similarity between [1,2,3] and [1,0,0]: {cosine_similarity(test_vec1, test_vec3):.4f} (expect ~0.27)")
print(f"\n✅ Test passed! Your implementation is correct.")

## Task 4: Build a Semantic Search Function

Now let's create a search function that finds the most relevant documents for a query.

In [ ]:
def semantic_search(query: str, top_k: int = 3) -> List[Tuple[int, float, str]]:
    """
    Find the most relevant documents for a given query.
    
    Args:
        query: Search query string
        top_k: Number of results to return
    
    Returns:
        List of (doc_index, similarity_score, document_text) tuples
    """
    # Step 1: Convert query to vector
    query_embedding = model.encode([query])[0]
    
    # Step 2: Calculate similarity with all documents
    similarities = []
    for i, doc_embedding in enumerate(document_embeddings):
        sim = cosine_similarity(query_embedding, doc_embedding)
        similarities.append((i, sim, documents[i]))
    
    # Step 3: Sort by similarity (highest first) and return top_k
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return similarities[:top_k]

# Test the search function
test_query = "How do neural networks learn from images?"
print(f"🔍 Query: '{test_query}'\n")
print("📊 Top 3 Results:\n")

results = semantic_search(test_query, top_k=3)
for rank, (doc_id, score, text) in enumerate(results, 1):
    print(f"{rank}. [Score: {score:.4f}] {text}\n")

## 🧪 Experiment: Compare Different Queries

Try different queries to see how semantic search works!

In [ ]:
# Try different queries
queries = [
    "What is machine learning?",
    "How do computers understand images?",
    "Training models without labels",
    "Language understanding by machines",
]

for query in queries:
    print(f"\n{'='*80}")
    print(f"🔍 Query: '{query}'")
    print(f"{'='*80}\n")
    
    results = semantic_search(query, top_k=2)
    for rank, (doc_id, score, text) in enumerate(results, 1):
        print(f"{rank}. [Score: {score:.4f}]\n   {text}\n")

## 📊 Visualization: Document Similarity Matrix

Let's visualize how similar all documents are to each other.

In [ ]:
# Calculate similarity matrix
n_docs = len(documents)
similarity_matrix = np.zeros((n_docs, n_docs))

for i in range(n_docs):
    for j in range(n_docs):
        similarity_matrix[i][j] = cosine_similarity(
            document_embeddings[i], 
            document_embeddings[j]
        )

# Plot heatmap
plt.figure(figsize=(12, 10))
plt.imshow(similarity_matrix, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Cosine Similarity')
plt.title('Document Similarity Matrix', fontsize=16, fontweight='bold')
plt.xlabel('Document Index')
plt.ylabel('Document Index')
plt.xticks(range(n_docs))
plt.yticks(range(n_docs))

# Add similarity values to cells
for i in range(n_docs):
    for j in range(n_docs):
        text = plt.text(j, i, f'{similarity_matrix[i, j]:.2f}',
                       ha="center", va="center", color="black", fontsize=8)

plt.tight_layout()
plt.show()

print("\n💡 Insights:")
print("   - Diagonal values are 1.0 (documents are identical to themselves)")
print("   - Brighter colors = higher similarity")
print("   - Look for clusters of similar documents!")

## 🎯 Challenge Exercises

Test your understanding with these challenges:

### Challenge 1: Implement Euclidean Distance

Besides cosine similarity, another common distance metric is Euclidean distance.

**Formula:** 
$$d(A, B) = \sqrt{\sum_{i=1}^{n}(A_i - B_i)^2}$$

In [ ]:
def euclidean_distance(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    Calculate Euclidean distance between two vectors.
    
    TODO: Implement this function
    Hint: Use np.sqrt() and np.sum()
    """
    # YOUR CODE HERE
    pass

# Test your implementation
# vec1 = np.array([1, 2, 3])
# vec2 = np.array([4, 5, 6])
# print(f"Euclidean distance: {euclidean_distance(vec1, vec2):.4f}")

### Challenge 2: Add Your Own Documents

Expand the knowledge base with your own documents and test queries!

In [ ]:
# TODO: Add 5 new documents about a topic of your choice
new_documents = [
    # Add your documents here
]

# Re-encode all documents
# all_documents = documents + new_documents
# all_embeddings = model.encode(all_documents)

# Test your new search!

### Challenge 3: Build a Keyword Filter

Combine semantic search with keyword filtering. Return only results that contain specific keywords.

In [ ]:
def hybrid_search(query: str, required_keywords: List[str], top_k: int = 3):
    """
    Semantic search with keyword filtering.
    
    TODO: Implement this function
    - First do semantic search
    - Then filter results to only include documents containing ALL required_keywords
    """
    # YOUR CODE HERE
    pass

## 🎓 Key Takeaways

### What You've Learned:

1. **Vector Embeddings**: Text can be represented as numerical vectors that capture semantic meaning

2. **Cosine Similarity**: Measures the angle between vectors, not their magnitude
   - Perfect for semantic similarity (direction matters more than length)
   - Range: -1 (opposite) to 1 (identical)

3. **Semantic Search**: Finds conceptually related content, not just keyword matches
   - Query: "How do computers see?" → Finds "Computer vision allows machines..."
   - Traditional search would miss this!

4. **The Foundation of RAG**: This exact pattern is used in production AI agents:
   - Vector Database stores embeddings
   - Query gets embedded
   - Similarity search retrieves context
   - LLM generates answer using retrieved docs

### Real-World Applications:

- **Chatbots**: Retrieve relevant knowledge before answering
- **Recommendation Systems**: Find similar products/content
- **Document Search**: Semantic search across large corpora
- **Question Answering**: RAG systems (Module 4!)

### Next Steps:

In Module 2, you'll explore how LLMs themselves work - the models that both generate embeddings AND produce text!

## 📚 Additional Resources

- [Sentence Transformers Documentation](https://www.sbert.net/)
- [Understanding Word Embeddings](https://jalammar.github.io/illustrated-word2vec/)
- [Vector Similarity Search Explained](https://www.pinecone.io/learn/vector-similarity/)
- [The Illustrated BERT](https://jalammar.github.io/illustrated-bert/)